In [1]:
#!pip install azure-cognitiveservices-vision-customvision
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
import numpy as np

# Create Project

In [2]:
ENDPOINT = "https://<resource-name>.cognitiveservices.azure.com/"

# Replace with a valid key
training_key = "<training-key>"
publish_iteration_name = "classifyBirdModel"
prediction_resource_id ="/subscriptions/<sub-id>/resourceGroups/<resource-group-name>/providers/Microsoft.CognitiveServices/accounts/<resource-name-Prediction>"
trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

# Create a new project
print ("Creating project...")

# Create the project based on if you need a default or compact model. Only the compact models can be exported, default is deployed with custom vision only.
#project = trainer.create_project("Bird Classification")
project = trainer.create_project(name="Bird Classification Compact", domain_id="General (compact)", classification_type="Multiclass", target_export_platforms="Basic platforms")
print("Project created!")

Creating project...
Project created!


# Create Tag list from file names

In [3]:
#Create Tag List from folders in bird directory
import os
os.chdir('./bird_photos/train')
tags = [name for name in os.listdir('.') if os.path.isdir(name)]
print(tags)

['American Crow', 'American Goldfinch (Female)', 'American Goldfinch (Male)', 'American Robin (Adult)', 'American Robin (Juvenile)', 'Blue Jay', 'Common Grackle', 'House Sparrow (Female)', 'House Sparrow (Male)', 'House Wren', 'Mourning Dove', 'Northern Cardinal (Adult Male)', 'Northern Cardinal (Female)', 'Red-tailed Hawk (Dark morph)', 'Red-tailed Hawk (Light morph immature)', 'Tufted Titmouse']


# Create tag in Azure

In [4]:
def createTag(tag):
    result = trainer.create_tag(project.id, tag)
    print(f'{tag} create with id: {result}')
    return result.id


# Create Image List to Upload

In [5]:
def createImageList(tag, tag_id):
    #set directory to current tag
    base_image_url = f"./{tag}/"
    photo_name_list = os.listdir(base_image_url)
    image_list = []
    for file_name in photo_name_list:
        with open(base_image_url+file_name, "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=base_image_url+file_name, contents=image_contents.read(), tag_ids=[tag_id]))
    return image_list

# Upload List method

In [6]:
def uploadImageList(image_list):  
    upload_result = trainer.create_images_from_files(project.id, images=image_list)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)

# Loop thru tags and call each function |to upload

In [7]:
#get images names from directory
for tag in tags:
    tag_id = createTag(tag)
    print(f"tag creation done with tag id {tag_id}")
    image_list = createImageList(tag, tag_id)
    print("image_list created with length " + str(len(image_list)))
    
    #break list into lists of 25 and upload in batches
    for i in range(0, len(image_list), 25):
        batch = image_list[i:i + 25]
        print(f'Upload started for batch {i} total items {len(batch)} for tag {tag}...')
        uploadImageList(batch)
        print(f"Batch {i} Image upload completed. Total uploaded {len(batch)} for tag {tag}")

American Crow create with id: {'additional_properties': {}, 'id': '5b72852d-cb9b-4bcf-8e0d-0df8d721a95d', 'name': 'American Crow', 'description': None, 'type': 'Regular', 'image_count': 0}
tag creation done with tag id 5b72852d-cb9b-4bcf-8e0d-0df8d721a95d
image_list created with length 84
Upload started for batch 0 total items 25 for tag American Crow...
Batch 0 Image upload completed. Total uploaded 25 for tag American Crow
Upload started for batch 25 total items 25 for tag American Crow...
Batch 25 Image upload completed. Total uploaded 25 for tag American Crow
Upload started for batch 50 total items 25 for tag American Crow...
Batch 50 Image upload completed. Total uploaded 25 for tag American Crow
Upload started for batch 75 total items 9 for tag American Crow...
Batch 75 Image upload completed. Total uploaded 9 for tag American Crow
American Goldfinch (Female) create with id: {'additional_properties': {}, 'id': '35faee15-b212-4f85-a5cc-90b68b0a9231', 'name': 'American Goldfinch (F

House Wren create with id: {'additional_properties': {}, 'id': '2d4e7965-fb0d-40c1-9d80-624239ee7363', 'name': 'House Wren', 'description': None, 'type': 'Regular', 'image_count': 0}
tag creation done with tag id 2d4e7965-fb0d-40c1-9d80-624239ee7363
image_list created with length 84
Upload started for batch 0 total items 25 for tag House Wren...
Batch 0 Image upload completed. Total uploaded 25 for tag House Wren
Upload started for batch 25 total items 25 for tag House Wren...
Batch 25 Image upload completed. Total uploaded 25 for tag House Wren
Upload started for batch 50 total items 25 for tag House Wren...
Batch 50 Image upload completed. Total uploaded 25 for tag House Wren
Upload started for batch 75 total items 9 for tag House Wren...
Batch 75 Image upload completed. Total uploaded 9 for tag House Wren
Mourning Dove create with id: {'additional_properties': {}, 'id': 'baf922d8-15d6-4a68-aa74-d37b9e480a9d', 'name': 'Mourning Dove', 'description': None, 'type': 'Regular', 'image_co

# Train Model

In [8]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(5)


Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed


In [9]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


In [10]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

prediction_key = "<prediction-key>"
prediction_endpoint ="https://<resource-name-prediction>.cognitiveservices.azure.com/"
# Now there is a trained endpoint that can be used to make a prediction
predictor = CustomVisionPredictionClient(prediction_key, endpoint=prediction_endpoint)

In [13]:
#get currect directory
os.getcwd()

'C:\\Code\\Demos\\Reactor AML Talk\\ImageScraperAndCustomVisionMagic\\bird_photos'

In [12]:
#move back
os.chdir("../")

In [14]:
#C:\Code\Demos\Reactor AML Talk\bird_photos\val\American Crow\bf579add7cbc400bbc6b374dff68d23c.jpg
val_image_url = f"./val/American Crow/bf579add7cbc400bbc6b374dff68d23c.jpg"

In [15]:
with open(val_image_url, "rb") as image_contents:
    results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))

	American Crow: 100.00%
	Common Grackle: 0.00%
	Blue Jay: 0.00%
	American Robin (Adult): 0.00%
	House Sparrow (Female): 0.00%
	Northern Cardinal (Female): 0.00%
	American Robin (Juvenile): 0.00%
	Red-tailed Hawk (Dark morph): 0.00%
	Northern Cardinal (Adult Male): 0.00%
	Mourning Dove: 0.00%
	Tufted Titmouse: 0.00%
	House Sparrow (Male): 0.00%
	House Wren: 0.00%
	Red-tailed Hawk (Light morph immature): 0.00%
	American Goldfinch (Female): 0.00%
	American Goldfinch (Male): 0.00%
